# Pattern 2

## Prepare Resource IDs

まずは作業中に利用する、ECR repository と S3 bucket の ID を変数に代入しておく。CDK でデプロイした CloudFormation stack を確認して、以下を編集して代入しておく。

In [ ]:
repository = '...'

In [ ]:
import sagemaker
bucket = sagemaker.Session().default_bucket()
bucket

また AWS account ID と region ID の情報も必要となるので、こちらも変数に代入しておく。

In [ ]:
import boto3
sts = boto3.client('sts')

account = sts.get_caller_identity()['Account']
account

In [ ]:
region = boto3.Session().region_name
region

## Prepare Model Artifact

モデルのアーティファクト（モデルの重みが保存されているファイル）を準備する。
このパターンでは、 NumPy の行列を利用する。

In [ ]:
import numpy as np
import tarfile

w = np.identity(2, dtype=np.float32) * 10
w

In [ ]:
np.save('model.npy', w)
with tarfile.open('model.tar.gz', 'w:gz') as f:
    f.add('model.npy')

In [ ]:
model_data = f's3://{bucket}/byoc-inference/numpy/model.tar.gz'

In [ ]:
!aws s3 cp model.tar.gz {model_data}

## Prepare Container Image

コンテナイメージを準備する。このパターンでは、ブログの通り 4 つの手順に分解される。

1. (省略可能) Handler の実装とコンテナイメージへのコピー
2. (省略可能) HandlerService の実装とコンテナイメージへのコピー
3. コンテナ起動時のエントリーポイントの実装とコンテナイメージへのコピー
4. コンテナイメージのビルド

まずは 3 である。

In [ ]:
%%writefile container_entry_point.py
from sagemaker_inference import model_server 

model_server.start_model_server()

続いては 4 である。Dockerfile は以下である。

In [ ]:
%%writefile Dockerfile
FROM python:3.8
WORKDIR /usr/src/app
RUN apt-get update && apt-get upgrade -y && apt-get install -y openjdk-17-jdk-headless
RUN pip install --no-cache-dir numpy multi-model-server sagemaker-inference
COPY container_entry_point.py ./
ENTRYPOINT ["python", "/usr/src/app/container_entry_point.py"]

これをビルドする。

In [ ]:
!docker image build . -t byoc-inference-pattern2

In [ ]:
!docker image ls

今ビルドしたコンテナイメージを、自身の AWS account の ECR registry の repostiry へプッシュする。まずは、認証情報を登録する。

In [ ]:
!aws ecr get-login-password | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com

In [ ]:
!docker image tag byoc-inference-pattern2 {account}.dkr.ecr.{region}.amazonaws.com/{repository}:pattern2

In [ ]:
!docker image ls

In [ ]:
!docker image push {account}.dkr.ecr.{region}.amazonaws.com/{repository}:pattern2

## Prepare Inference Code

推論コードを準備する。ブログにも記載のように別のディレクトリを作成する必要があることに注意である。

In [ ]:
!mkdir -p code

In [ ]:
%%writefile ./code/entry_point.py
from pathlib import Path
import numpy as np
from sagemaker_inference import content_types, decoder, default_inference_handler, encoder

def model_fn(model_dir, context=None):
    return np.load(str(Path(model_dir) / 'model.npy'))

def input_fn(input_data, content_type, context=None):
    print(input_data)
    print(content_type)
    return decoder.decode(input_data, content_type)

def predict_fn(data, model, context=None):
    return model @ data
    
def output_fn(prediction, accept, context=None):
    return encoder.encode(prediction, accept)

## Deploy and Invoke SageMaker Endpoint

モデルアーティファクト、コンテナイメージ、推論コードの準備が整ったので、これらを用いて SageMaker endpoint をデプロイする。

In [ ]:
import sagemaker
from sagemaker import Model

model = Model(
    model_data=model_data,
    image_uri=f'{account}.dkr.ecr.{region}.amazonaws.com/{repository}:pattern2',
    entry_point='entry_point.py',
    source_dir='./code',
    role=sagemaker.get_execution_role(),
    predictor_cls=sagemaker.predictor.Predictor,
)

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

In [ ]:
predictor.serializer = sagemaker.serializers.NumpySerializer(dtype=np.float32)
predictor.deserializer = sagemaker.deserializers.NumpyDeserializer(dtype=np.float32)

In [ ]:
predictor.predict(np.array([1, 2]))

以上でパターン 2 は完了である。

## Clean Up

In [ ]:
predictor.delete_endpoint()